In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import regex as re
import plotly.express as px
import requests
from bs4 import BeautifulSoup as BS
from io import StringIO
import statsmodels
%matplotlib inline

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Read in data on well permits
Data objtained from TN Department of Enviornment & Conservation
http://tdec.tn.gov:8080/pls/enf_reports/f?p=9034:34300:0::NO::: 

In [4]:
wells = pd.read_csv('../data/oil_and_gas_well_permits.csv')

In [5]:
wells.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4677 entries, 0 to 4676
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   County                    4677 non-null   object 
 1   EFO Name                  4677 non-null   object 
 2   Permit Date               4677 non-null   object 
 3   Permit No                 4677 non-null   int64  
 4   Latitude                  4677 non-null   float64
 5   Longitude                 4677 non-null   float64
 6   Formation at Total Depth  4522 non-null   object 
 7   Purpose af Well           4523 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 292.4+ KB


In [6]:
#transform date object to datetime
wells['Permit Date'] = pd.to_datetime(wells['Permit Date'])

In [7]:
wells.columns

Index(['County', 'EFO Name', 'Permit Date', 'Permit No', 'Latitude',
       'Longitude', 'Formation at Total Depth', 'Purpose af Well'],
      dtype='object')

In [9]:
#rename colummns
wells.columns=['county', 'efo_name', 'permit_date', 'permit_no', 'latitude',
       'longitude', 'geoformation_at_well_depth', 'well_purpose']

In [162]:
wells.head()

,county,efo_name,permit_date,permit_no,latitude,longitude,geoformation_at_well_depth,well_purpose
0,Clay,Cookeville,2019-12-26,13065,36.546333,-85.308694,Knox Group,Oil
1,Overton,Cookeville,2019-12-26,13063,36.392583,-85.205306,Knox Group,Oil
2,Fentress,Cookeville,2019-12-26,13064,36.418806,-84.978333,Chattanooga Shale,Oil And Gas
3,Pickett,Cookeville,2019-12-23,13062,36.547861,-85.278444,Knox Group,Oil
4,Clay,Cookeville,2019-12-23,13060,36.545361,-85.285500,Knox Group,Oil


In [10]:
#remove NAN values to avoid errors in running Plotly
wells = wells.dropna(how='any')

In [12]:
#Check shape against original df to see how many were dropped
wells.shape

(4522, 8)

### What is the general trend of well permits since 1990? 

--> Marked increase in early 2000s through 2008.

In [14]:
fig = px.histogram(wells, x="permit_date", histfunc='count',
                   #color = 'geoformation_at_well_depth', 
                   title='TN Well Permits Over Time')
fig.show()

### Where are the locations of these permits?

In [28]:
df=wells
fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='geoformation_at_well_depth', 
                        color_discrete_sequence= px.colors.sequential.Plasma,zoom=3, height=700, width=1500,
                       title= "Oil & Gas Well Permits in TN - 1900-2020")
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=6, mapbox_center = {"lat": 35.9917, "lon": -84.3364},)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [25]:
#Create a year column from the datetime values of 'permit_year' to plot with animation
wells['permit_year']= wells['permit_date'].dt.year

### Where are permits clustered by year?

In [26]:
df=wells.sort_values(by='permit_date', ascending = True)
fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='geoformation_at_well_depth', 
                        animation_frame="permit_year", animation_group="permit_year",
                color_discrete_sequence= px.colors.sequential.Plasma,
                        hover_data=wells,zoom=3, height=700)
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=6, mapbox_center = {"lat": 35.9917, "lon": -84.3364},)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()